# Prerequisites : 

### _Only in Local Environment set the following ENVIRONMENTAL Variables_.

- DI_CLUSTER_URL="https://xxxxxxxxxxxxxxxxxx.sapdatahub.com:xxxx" <br>
- DI_TENANT="di tenant" <br>
- DI_USERNAME="DI logon User name" <br>
- DI_PASSWORD="DI logon Password" <br>
- DI_SCENARIO_ID="xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"<br>

_The Notebooks in DI will get the connection details automatically._

### _Optional Environmental variable_

- DI_PIPELINE_ID="xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"<br>

Pipeline can be set programmatically using `api.set_pipeline_by_id('xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx')`

#### <font color="red"> Any operations to DI needs the DI-MODE Enabled.</font>


It can be done via `api.set_di_mode_on()`. It can only be switched on if the **PIPELINE ID** is already SET or By creating a complete new PIPELINE via `api.create_pipeline()`. It can be switched off via `api.set_di_mode_off()`. 



### Needed Imports

In [1]:
from nb2di import api
from nb2di import Operator, PortKind, ContentType
from nb2di import Port, PortTemplate, ConnectionTemplate, ConnectionTemplateSourceToTarget

connecting to DI...
connected in 1.5515708923339844 seconds


# Example 1 : Injecting Notebook Code to existing Pipeline to scriptable Operator in DI.

#### 1. Set DI Pipeline Context

In [45]:
api.set_pipeline_by_id('c1450391-50d8-46e4-b1ed-6d1b11f3c624')

Current DI Pipeline Set 36.49886989593506


('c1450391-50d8-46e4-b1ed-6d1b11f3c624', 'base')

#### 2. Define some function which needs to be injected in operator

**instance_id** _is the ID of the operator._

In [5]:

@api.operator(instance_id='python3operator1')
def some_function():
    
    some_var = "hello"
    
    # log info in DI
    api.logger.info("Hello")
    
    # send it to some outports
    api.send("metrics", api.Message("hello", some_var))

#### 3. Test the function locally 

In [7]:
some_function()

Arguments ('Hello',) 
Keyed-Arguments {}
Arguments ('hello', 'hello') 
Keyed-Arguments {}
Arguments ('metrics', None) 
Keyed-Arguments {}


#### 4. Set DI Mode to ON

In [8]:
api.set_di_mode_on()

#### 5. Inject to DI

In [9]:
# run the function again which injects it to DI Operator.
api.set_port_callback("input", some_function)


some_function()

#### 6. Optional: Execute the Graph Pipeline 

In [9]:
#api.execute_pipeline()

# Example 2 : Create New Operator in Existing Pipeline and Inject Code

#### 1. Optional : get component_name of operator

In [10]:
api.get_operator_component_name(name='python3')

['com.sap.system.python3Operator',
 'com.sap.util.python3TypeToBlob',
 'com.sap.util.python3TypeToString']

#### 2. Define some function which needs to be injected in operator.

**component_name** : _Operators component name which needs to be created_.


In [11]:
@api.operator(component_name='com.sap.system.python3Operator')
def some_function():
    
    some_var = "test"
    
    # log info in DI
    api.logger.info("Hello")
    
    # send it to some outports
    api.send("metrics", api.Message("hello", some_var))

#### 4. Set DI Mode to ON

In [12]:
api.set_di_mode_on()

#### 5. Create Operator and Inject to DI

In [13]:
# call the function...
some_function()

#### 6. Optional: Execute the Graph Pipeline 

In [14]:
#api.execute_pipeline()

# Example 3 : Create new PORT(s) to existing Operator

#### 1. Needed Import

In [14]:
from nb2di import PortTemplate, PortKind, ContentType

#### 2. Set DI Mode to ON

In [15]:
# if not set before..
api.set_di_mode_on()

#### 3. Create PortTemplate instance
- name: unique name
- kind: PortKind.INPUT or PortKind.OUTPUT
- content_type: ContentType.MESSAGE or ContentType.STRING or etc...
- target: instance of ConnectionTemplate

**NOTE**: _target is used to set connections automatically to the created port to target operator/port._

In [16]:
op_1 = PortTemplate(name="testoutport", kind=PortKind.OUTPUT, content_type=ContentType.MESSAGE)

In [17]:
op_2 = PortTemplate(name="testoutport2", kind=PortKind.OUTPUT, content_type=ContentType.MESSAGE)

#### 4. Set Ports to Operator

_ports is a list., allowing multiple ports created at once._

In [18]:
api.add_port_to_operator(instance_id='python3Operator3', ports=[op_1, op_2])

**NOTE**: _Sometimes., Ports without connections are not visible in Graph Modeler._

#### 5. [Optional] Ports created successfully can be seen in the operator instance.

In [19]:
from nb2di import Operator

In [20]:
opr = Operator(instance_id='python3Operator3')

In [21]:
# should contain the list of outports
print(opr.ports.outports.list_port())

# should contain the list of inports
print(opr.ports.inports.list_port())

[<sapdi.internal.modeler.port_info.PortInfo object at 0x13817a160>, <sapdi.internal.modeler.port_info.PortInfo object at 0x13817a208>]
[]


# Example 4 : Create Connections between from Existing Operators / Ports.

#### 1. Needed Imports

In [22]:
from nb2di import Operator, ConnectionTemplateSourceToTarget

#### 2. Set DI Mode to ON

In [23]:
# if not set before..
api.set_di_mode_on()

#### 3. Get Operator's instance

In [24]:
# source operator
src_opr = Operator(instance_id='python3Operator3')

#target operator
tgt_opr = Operator(instance_id='submitmetrics1')

#### 4. check the ports exists

In [25]:
print("source :" , src_opr.ports.outports.testoutport)
print("target :" , tgt_opr.ports.inports.metrics)

source : <nb2di.port.Port object at 0x10934d240>
target : <nb2di.port.Port object at 0x13817a2e8>


#### 5. create connection template

In [26]:
conn = ConnectionTemplateSourceToTarget(src=src_opr.ports.outports.testoutport, tgt=tgt_opr.ports.inports.metrics)

#### 6. create connection
- **connections** is list where multiple connections created together.

In [27]:
api.add_connections_to_port(connections=[conn])

# Example 5 : Create New Pipeline Graph

#### 1. Set DI Mode to ON

In [28]:
# if not set before..
api.set_di_mode_on()

#### 2. Optional: Get Pipeline templates..
_only if you want to know the technical names of the template._

In [29]:
api.get_pipeline_template_name(name='PYTHOn')

['com.sap.dsp.templates.python_producer_template',
 'com.sap.dsp.templates.python_consumer_template']

#### 3. Create Pipeline
- name: Optional
- desc: Optional
- template: Optional ( Example: com.sap.dsp.templates.python_producer_template )

**Note**: _The created pipeline automatically set to current pipeline context. <br> In-case you want to work with different pipeline then `api.set_pipeline_by_id('xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx')`._


In [30]:
# Empty pipeline autogenerated names
api.create_pipeline(name="test-1")

Current DI Pipeline Set 5.623921155929565


'd9111d03-3253-404e-b5dd-777e2c544aff'

In [31]:
# Template pipeline
api.create_pipeline(name="test-2", template='com.sap.dsp.templates.python_producer_template')

Current DI Pipeline Set 59.77549910545349


'd1ed2adf-e088-4528-aa26-7cb5b6455549'

# Example 6 : Create Everything from Scratch via Notebook ( Pipeline, operators , connections ) all at once.

In this example., We create.,
- Python Operator
    - test  (OUTPORT)
- Submit Metrics Operator
    - metrics (INPORT)   -- generated automatically by operator.
    - response (OUTPORT) -- generated automatically by operator.
- Graph Terminator Operator
    - stop (INPORT)
    
And **create connections** to the ports like.,

python operator's **test** port -> submit metrics **metrics** port <br>
submit metrics **response** port -> graph terminator **stop** port

#### 1. Needed Imports

In [33]:
from nb2di import Operator, ConnectionTemplateSourceToTarget
from nb2di import PortTemplate, PortKind, ContentType
from nb2di import api

#### 2. Create New Pipeline

In [34]:
api.create_pipeline()

Current DI Pipeline Set 2.7942211627960205


'02647ff7-c395-4390-aa89-214da469dc12'

In [35]:
api.get_current_pipeline_id()

('02647ff7-c395-4390-aa89-214da469dc12', 'gen-pipeline-1')

#### 3. Create Operators and Ports

In [36]:
# create port template
python_opr_port = PortTemplate(name="test", kind=PortKind.OUTPUT, content_type=ContentType.MESSAGE)

# create python operator
python_opr = Operator(component_name="com.sap.system.python3Operator", create_ports= [python_opr_port])

In [37]:
api.get_operator_component_name(name="submit")

['com.sap.hadoop.submitJob', 'com.sap.livy.submit', 'com.sap.ml.submitMetrics']

In [38]:
metrics_opr = Operator(component_name="com.sap.ml.submitMetrics")

In [39]:
api.get_operator_component_name(name="terminator")

['com.sap.dh.terminator', 'com.sap.util.graphTerminator']

In [40]:
# create port template
terminator_opr_port = PortTemplate(name="stop", kind=PortKind.INPUT, content_type=ContentType.ANY)

terminator_opr = Operator(component_name="com.sap.util.graphTerminator", create_ports=[terminator_opr_port])

In [41]:
## check the ports
print(python_opr.ports.outports.test.name)
print(metrics_opr.ports.inports.metrics.name)
print(metrics_opr.ports.outports.response.name)
print(terminator_opr.ports.inports.stop.name)

test
metrics
response
stop


#### 4. Create Connections from Operator's Port

In [42]:
python_opr.ports.outports.test.connection.add_connection(metrics_opr.ports.inports.metrics)

In [43]:
metrics_opr.ports.outports.response.connection.add_connection(terminator_opr.ports.inports.stop)

#### 5. Save Graph

**Note**: save graph can be called from any operator instance.

In [44]:
python_opr.save_graph()